In [1]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
import pandas as pd
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.calc.mat import LabeledArray, combine

from utils import *
import matplotlib.pyplot as plt

from pandas import read_csv
import scipy.stats as stats
import joblib

from scipy.ndimage import label
from scipy.stats import norm

import json
import pickle

# rsa toolbox imports
from rsatoolbox.io.mne import read_epochs
from rsatoolbox.data.ops import merge_datasets
from rsatoolbox.rdm import calc_rdm_movie
from rsatoolbox.rdm.calc import _parse_input
from rsatoolbox.util.build_rdm import _build_rdms
from rsatoolbox.rdm import compare
from rsatoolbox.vis import show_rdm
from rsatoolbox.vis.timecourse import plot_timecourse

from os.path import join, expanduser, basename
import glob, json
import numpy, tqdm, mne, pandas
import rsatoolbox
from sklearn.preprocessing import MinMaxScaler
from matplotlib import pyplot

from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup
from jim_decoding_functions import *


['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']



Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
# subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103', 'D0107A', 'D0110']
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# subjects = ['D0100', 'D0102', 'D0103']
# load in subjects electrodes to rois dict. If it doesn't already exist, make it and then load it.
filename = 'subjects_electrodestoROIs_dict.json'
subjects_electrodestoROIs_dict = make_or_load_subjects_electrodes_to_rois_dict(filename, subjects)

Attempting to load the subjects' electrodes-to-ROIs dictionary...
Loaded data from subjects_electrodestoROIs_dict.json
Dictionary loaded successfully. Ready to proceed!


In [3]:
stimulus_experiment_conditions = {
    "Stimulus/i25.0/s25.0": {
        "BIDS_events": "Stimulus/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/s75.0": {
        "BIDS_events": "Stimulus/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/s25.0": {
        "BIDS_events": "Stimulus/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/s75.0": {
        "BIDS_events": "Stimulus/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/i25.0/r25.0": {
        "BIDS_events": "Stimulus/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i25.0/r75.0": {
        "BIDS_events": "Stimulus/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/i75.0/r25.0": {
        "BIDS_events": "Stimulus/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/i75.0/r75.0": {
        "BIDS_events": "Stimulus/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/s25.0": {
        "BIDS_events": "Stimulus/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/s75.0": {
        "BIDS_events": "Stimulus/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/s25.0": {
        "BIDS_events": "Stimulus/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/s75.0": {
        "BIDS_events": "Stimulus/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Stimulus/c25.0/r25.0": {
        "BIDS_events": "Stimulus/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c25.0/r75.0": {
        "BIDS_events": "Stimulus/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Stimulus/c75.0/r25.0": {
        "BIDS_events": "Stimulus/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Stimulus/c75.0/r75.0": {
        "BIDS_events": "Stimulus/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

# congruency_conditions = {
#     "Stimulus_c": {
#         "BIDS_events": ["Stimulus/c25/s25", "Stimulus/c25/s75", "Stimulus/c75/s25", "Stimulus/c75/s75", "Stimulus/c25/r25", "Stimulus/c25/r75", "Stimulus/c75/r25", "Stimulus/c75/r75"],
#         "congruency": "c"
#     }
# }

stimulus_conditions = {
    "Stimulus/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Stimulus/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Stimulus/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

response_experiment_conditions = {
    "Response/i25.0/s25.0": {
        "BIDS_events": "Response/i25.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i25.0/s75.0": {
        "BIDS_events": "Response/i25.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i75.0/s25.0": {
        "BIDS_events": "Response/i75.0/s25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/i75.0/s75.0": {
        "BIDS_events": "Response/i75.0/s75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/i25.0/r25.0": {
        "BIDS_events": "Response/i25.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i25.0/r75.0": {
        "BIDS_events": "Response/i25.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/i75.0/r25.0": {
        "BIDS_events": "Response/i75.0/r25.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/i75.0/r75.0": {
        "BIDS_events": "Response/i75.0/r75.0",
        "congruency": "i",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c25.0/s25.0": {
        "BIDS_events": "Response/c25.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c25.0/s75.0": {
        "BIDS_events": "Response/c25.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c75.0/s25.0": {
        "BIDS_events": "Response/c75.0/s25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "25%"
    },
    "Response/c75.0/s75.0": {
        "BIDS_events": "Response/c75.0/s75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "s",
        "switchProportion": "75%"
    },
    "Response/c25.0/r25.0": {
        "BIDS_events": "Response/c25.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c25.0/r75.0": {
        "BIDS_events": "Response/c25.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "75%",
        "switchType": "r",
        "switchProportion": "75%"
    },
    "Response/c75.0/r25.0": {
        "BIDS_events": "Response/c75.0/r25.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "25%"
    },
    "Response/c75.0/r75.0": {
        "BIDS_events": "Response/c75.0/r75.0",
        "congruency": "c",
        "congruencyProportion": "25%",
        "switchType": "r",
        "switchProportion": "75%"
    }
}

response_conditions = {
    "Response/BigLetters/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskg",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetters/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetterh/Taskl",
        "bigLetter": "s",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetters/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskg",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetters/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetters/SmallLetters/Taskl",
        "bigLetter": "s",
        "smallLetter": "s",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetterh/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskg",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetterh/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetterh/Taskl",
        "bigLetter": "h",
        "smallLetter": "h",
        "task": "l"
    },
    "Response/BigLetterh/SmallLetters/Taskg": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskg",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "g"
    },
    "Response/BigLetterh/SmallLetters/Taskl": {
        "BIDS_events": "Response/BigLetterh/SmallLetters/Taskl",
        "bigLetter": "h",
        "smallLetter": "s",
        "task": "l"
    }
}

In [ ]:
task='GlobalLocal'

conditions = stimulus_experiment_conditions # toggle
# conditions = stimulus_conditions # toggle

if conditions == stimulus_conditions or stimulus_experiment_conditions:
    epochs_root_file = "Stimulus_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10"
elif conditions == response_conditions or response_experiment_conditions:
    epochs_root_file = "Response_0.5sec_within1sec_randoffset_preStimulusBase_decFactor_8_outliers_10"
else:
    raise ValueError("Unknown condition type.")

condition_names = list(conditions.keys()) # get the condition names as a list

# This breaks if just_HG_ev1_rescaled is set to False currently. Fix this! 8/11
subjects_mne_objects = create_subjects_mne_objects_dict(subjects=subjects, epochs_root_file=epochs_root_file, conditions=conditions, task="GlobalLocal", just_HG_ev1_rescaled=True, acc_trials_only=True)

Loading data for subject: D0057


In [121]:
condition_names = list(conditions.keys()) # get the condition names as a list
condition_names_bids = [condition['BIDS_events'] for condition in conditions.values()] # get the condition names in bids format
condition_names

['Stimulus/BigLetters/SmallLetterh/Taskg',
 'Stimulus/BigLetters/SmallLetterh/Taskl',
 'Stimulus/BigLetters/SmallLetters/Taskg',
 'Stimulus/BigLetters/SmallLetters/Taskl',
 'Stimulus/BigLetterh/SmallLetterh/Taskg',
 'Stimulus/BigLetterh/SmallLetterh/Taskl',
 'Stimulus/BigLetterh/SmallLetters/Taskg',
 'Stimulus/BigLetterh/SmallLetters/Taskl']

get significant channels

In [122]:
sig_chans_per_subject = get_sig_chans_per_subject(subjects, epochs_root_file, task='GlobalLocal', LAB_root=None)

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0069
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0094
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


In [167]:
# Define your ROIs dictionary and other parameters
rois_dict = {
    'lpfc': ["G_front_inf-Opercular", "G_front_inf-Orbital", "G_front_inf-Triangul", "G_front_middle", "G_front_sup", "Lat_Fis-ant-Horizont", "Lat_Fis-ant-Vertical", "S_circular_insula_ant", "S_circular_insula_sup", "S_front_inf", "S_front_middle", "S_front_sup"],
    'v1': ["G_oc-temp_med-Lingual", "S_calcarine", "G_cuneus"],
    'occ': ["G_cuneus", "G_and_S_occipital_inf", "G_occipital_middle", "G_occipital_sup", "G_oc-temp_lat-fusifor", "G_oc-temp_med-Lingual", "Pole_occipital", "S_calcarine", "S_oc_middle_and_Lunatus", "S_oc_sup_and_transversal", "S_occipital_ant"],
    'occ_filtered': [],
    'occ_best_filtered': []
}

rois = list(rois_dict.keys())
# Assuming you have subjects_electrodestoROIs_dict and sig_chans_per_subject dictionaries
electrodes_per_subject_roi, sig_electrodes_per_subject_roi, = make_sig_electrodes_per_subject_and_roi_dict(
    rois_dict, subjects_electrodestoROIs_dict, sig_chans_per_subject
)

For subject D0057, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI6', 'RAI12', 'RAI13', 'RAI14', 'RAI15', 'RAI16', 'RPI15', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12', 'RAMF13', 'RAMF14', 'RAIF11', 'RAIF12', 'RAIF13', 'RAIF14']
For subject D0059, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF10', 'LMMF12', 'LPSF16']
For subject D0063, G_front_inf-Opercular, G_front_inf-Orbital, G_front_inf-Triangul, G_front_middle, G_front_sup, Lat_Fis-ant-Horizont, Lat_Fis-ant-Vertical, S_circular_insula_ant, S_circular_insula_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LOF

let's manually make the occ filtered sig electrodes

In [168]:
sig_electrodes_per_subject_roi['occ_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': ['RO1', 'RO10'], #RO10 is iffy, big drop from fix onset
    'D0077': ['ROPM1', 'ROPM8'],
    'D0090': ['RTPO1', 'RTPI1'],
    'D0100': ['LOAI12', 'LOAI13', 'LOAI14', 'LOAI15', 'LOMI9', 'LOMI11', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9', 'LPPI7', 'LPPI8', 'LPPI9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

and now make the best of the best

In [169]:
sig_electrodes_per_subject_roi['occ_best_filtered'] = {
    'D0057': [],
    'D0059': [],
    'D0071': [],
    'D0077': [],
    'D0090': [],
    'D0100': ['LOAI12', 'LOMI9', 'LOPI8', 'LOPI9', 'LOPM8', 'LOPM9'],
    'D0102': ['RTPI1'],
    'D0103': ['LTPI2', 'LTPI3', 'LTPI4']
}

In [126]:
# Specify the file path where you want to save the dictionary
save_path = 'sig_electrodes_per_subject_roi.json'

# Use json to save the dictionary
with open(save_path, 'w') as file:
    json.dump(sig_electrodes_per_subject_roi, file, indent=4)

print(f"Dictionary saved to {save_path}")

Dictionary saved to sig_electrodes_per_subject_roi.json


get electrode counts for each roi

In [127]:
total_electrodes_info = calculate_total_electrodes(sig_electrodes_per_subject_roi, electrodes_per_subject_roi)
for roi, counts in total_electrodes_info.items():
    print(f"Total number of {roi} electrodes across all subjects:", counts['total_electrodes'])
    print(f"Total number of significant {roi} electrodes across all subjects:", counts['total_significant_electrodes'])


Total number of lpfc electrodes across all subjects: 164
Total number of significant lpfc electrodes across all subjects: 59
Total number of v1 electrodes across all subjects: 19
Total number of significant v1 electrodes across all subjects: 11
Total number of occ electrodes across all subjects: 76
Total number of significant occ electrodes across all subjects: 51
Total number of occ_filtered electrodes across all subjects: 0
Total number of significant occ_filtered electrodes across all subjects: 23
Total number of occ_best_filtered electrodes across all subjects: 0
Total number of significant occ_best_filtered electrodes across all subjects: 10


check if any subjects have a weird sampling rate

In [203]:
# Assuming 'subjects_mne_objects' is your dictionary containing MNE objects for each subject
sampling_rate = 256
subject_rates = check_sampling_rates(subjects_mne_objects, expected_sampling_rate=sampling_rate)


Subject D0057 has the expected sampling rate: 256.0 Hz.
Subject D0059 has the expected sampling rate: 256.0 Hz.
Subject D0063 has the expected sampling rate: 256.0 Hz.
Subject D0065 has the expected sampling rate: 256.0 Hz.
Subject D0069 has the expected sampling rate: 256.0 Hz.
Subject D0071 has the expected sampling rate: 256.0 Hz.
Subject D0077 has the expected sampling rate: 256.0 Hz.
Subject D0090 has the expected sampling rate: 256.0 Hz.
Subject D0094 has the expected sampling rate: 256.0 Hz.
Subject D0100 has the expected sampling rate: 256.0 Hz.
Subject D0102 has the expected sampling rate: 256.0 Hz.
Subject D0103 has the expected sampling rate: 256.0 Hz.


create a dat dict like in the Temporal example from rsatoolbox 7/19  
#### Data Dictionary (dat) Structure
- **roi1**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roi2**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample
- **roiX**
  - **data**: 3D data array for this ROI
  - **channel_names**: List of significant channels for this ROI
  - **cond_names**: Dictionary where keys are condition names and values are integer indices
  - **cond_idx**: 1D array where each entry is an integer index corresponding to a cond_name
  - **times**: 1D array of times, corresponding to each sample


        




In [129]:
# Example call to the function (you need to replace the arguments with actual data)
# dat = prepare_data_for_temporal_dataset(subjects_mne_objects, condition_names, rois, subjects, sig_electrodes_per_subject_roi)

make roi_labeled_arrays, a dict where the keys are rois and the values are LabeledArrays. Index the same way as a nested dict. Use .labels to get labels from current level.


#### roi_labeled_arrays structure
- **roi1**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roi2**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.

- **roiX**: ROI name, string
  - **conditions**: condition name
    - **trials**: This is the maximal number of trials across subjects for any condition, filled with nans for subjects who don't have this many trials
      - **channels**: This is the number of channels in the roi, each channel is labeled as subject-channel name. Concatenated across subjects.
        - **samples**: 1 sample as a float. This is the time for this sample.


some helper functions

In [130]:
def get_max_trials_per_condition(
    subjects_mne_objects, condition_names, subjects,
    sig_electrodes_per_subject_roi, roi, obs_axs
):
    """
    Find the maximum number of trials per condition across all subjects for a given ROI,
    and identify which subject(s) have that maximum number of trials.
    """
    max_trials_per_condition = {condition: 0 for condition in condition_names}
    max_trials_subject_per_condition = {condition: [] for condition in condition_names}

    for sub in subjects:
        sig_electrodes = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
        if not sig_electrodes:
            continue
        for condition_name in condition_names:
            # Check if the subject has data for this condition
            if condition_name not in subjects_mne_objects[sub]:
                continue
            epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_power_rescaled'].copy().pick(sig_electrodes)
            epochs_data = epochs.get_data(copy=True)
            n_trials = epochs_data.shape[obs_axs]
            if n_trials > max_trials_per_condition[condition_name]:
                max_trials_per_condition[condition_name] = n_trials
                max_trials_subject_per_condition[condition_name] = [sub]
            elif n_trials == max_trials_per_condition[condition_name]:
                max_trials_subject_per_condition[condition_name].append(sub)
    return max_trials_per_condition, max_trials_subject_per_condition

def make_subject_labeled_array(
    sub, subjects_mne_objects, condition_names, sig_electrodes_per_subject_roi,
    roi, max_trials_per_condition, obs_axs, chans_axs, time_axs, rng
):
    """
    Process data for a subject in a given ROI.

    This function performs the following steps for the given subject:
    1. Retrieves the significant electrodes in the specified ROI for the subject.
    2. For each condition:
        a. Extracts the epoch data for the significant electrodes.
        b. Randomizes the trial order using the provided random state.
        c. Pads the data with NaNs to match the maximum number of trials for that condition.
    3. Collects the processed data for all conditions into a dictionary.
    4. Creates a LabeledArray from the processed data, assigning appropriate channel and time labels.

    Returns a LabeledArray for the subject.
    """
    sig_electrodes = sig_electrodes_per_subject_roi.get(roi, {}).get(sub, [])
    if not sig_electrodes:
        return None

    subject_nested_dict = {}

    # Get channel names for this subject's ROI
    sub_channel_names = [f"{sub}-{sig_electrode}" for sig_electrode in sig_electrodes]

    # Loop through each condition
    for condition_name in condition_names:
        # Extract the epoch data for the current condition and subject
        epochs = subjects_mne_objects[sub][condition_name]['HG_ev1_power_rescaled'].copy().pick(sig_electrodes)
        epochs_data = epochs.get_data(copy=True)

        # Randomize the trial order
        n_trials = epochs_data.shape[obs_axs]
        print(f'in roi {roi}, subject {sub} has {n_trials} trials for condition {condition_name}')
        trial_indices = np.arange(n_trials)
        rng.shuffle(trial_indices)
        epochs_data = epochs_data.take(trial_indices, axis=obs_axs)

        # Get the target number of trials for padding
        max_trials = max_trials_per_condition[condition_name]

        # Pad with NaNs if necessary
        if n_trials < max_trials:
            padded_shape = list(epochs_data.shape)
            padded_shape[obs_axs] = max_trials
            padded_data = np.full(padded_shape, np.nan)
            indexer = [slice(None)] * epochs_data.ndim
            indexer[obs_axs] = slice(0, n_trials)
            padded_data[tuple(indexer)] = epochs_data
        else:
            padded_data = epochs_data

        subject_nested_dict[condition_name] = padded_data

    # Get time labels
    times = epochs.times
    str_times = [str(time) for time in times]
    np_array_str_times = np.array(str_times)

    # Create a LabeledArray for the subject
    subject_labeled_array = create_subject_labeled_array_from_dict(
        subject_nested_dict, sub_channel_names, np_array_str_times, chans_axs, time_axs
    )

    # Print the shape and time axis labels
    print(f"Subject {sub}, ROI {roi}, LabeledArray shape: {subject_labeled_array.shape}")
    time_axis_size = subject_labeled_array.shape[time_axs+1] # Adjusted time axis index

    return subject_labeled_array

def create_subject_labeled_array_from_dict(
    subject_nested_dict, sub_channel_names, np_array_str_times, chans_axs, time_axs
):
    """
    Create a LabeledArray for a subject with given data and labels.
    This accounts for the conditions axis by incrementing the channels and time axes by 1.
    """
    subject_labeled_array = LabeledArray.from_dict(subject_nested_dict)
    # Adjust axes indices due to the added conditions axis
    subject_labeled_array.labels[chans_axs + 1].values = sub_channel_names  # Channels axis
    subject_labeled_array.labels[time_axs + 1].values = np_array_str_times  # Time axis
    return subject_labeled_array

def concatenate_subject_labeled_arrays(
    roi_labeled_array, subject_labeled_array, chans_axs
):
    """
    Concatenate a subject's LabeledArray to the ROI's LabeledArray along the channels axis.
    Adjusts for the conditions axis by incrementing chans_axs by 1.
    """
    concatenation_axis = chans_axs + 1  # Adjusted channels axis index
    if roi_labeled_array is None:
        return subject_labeled_array
    else:
        return roi_labeled_array.concatenate(subject_labeled_array, axis=concatenation_axis)

main function

In [131]:
def put_data_in_labeled_array_per_roi_subject(
    subjects_mne_objects, condition_names, rois, subjects,
    sig_electrodes_per_subject_roi, obs_axs=0, chans_axs=1, time_axs=2,
    random_state=None
):
    """
    Organize the MNE data into separate LabeledArrays for each ROI and subject,
    with randomized trial ordering within each subject before concatenation.
    Concatenates subject data along the channels axis.

    Parameters:
    - subjects_mne_objects: Dictionary of MNE objects, structured as {subject: {condition: MNE epoch objects}}
    - condition_names: List of condition names.
    - rois: List of region of interest (ROI) names.
    - subjects: List of subjects.
    - sig_electrodes_per_subject_roi: Dictionary mapping ROIs to subjects and their corresponding electrodes.
    - obs_axs: The trials dimension (ignoring the conditions dimension for now)
    - chans_axs: The channels dimension
    - time_axs: The time dimension
    - random_state: Optional; an integer seed, NumPy RandomState, or None for random shuffling.

    Returns:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.
                          Each LabeledArray has dimensions: [Conditions, Trials, Channels, Timepoints]
    """
    # Set up the random state
    rng = np.random.RandomState(random_state)
    roi_labeled_arrays = {}

    # Loop through each ROI
    for roi in rois:
        # First pass: Find the max number of trials per condition across all subjects
        max_trials_per_condition, max_trials_subject_per_condition = get_max_trials_per_condition(
            subjects_mne_objects, condition_names, subjects,
            sig_electrodes_per_subject_roi, roi, obs_axs
        )

        # Print out the subjects with maximum trials
        print(f"ROI '{roi}': Maximum trials per condition:")
        for condition_name in condition_names:
            max_trials = max_trials_per_condition[condition_name]
            subjects_with_max_trials = max_trials_subject_per_condition[condition_name]
            print(f"  Condition '{condition_name}': {max_trials} trials from subjects {subjects_with_max_trials}")
            
        # Initialize the ROI LabeledArray
        roi_labeled_array = None

        # Second pass: Process each subject's data
        for sub in subjects:
            subject_labeled_array = make_subject_labeled_array(
                sub, subjects_mne_objects, condition_names, sig_electrodes_per_subject_roi,
                roi, max_trials_per_condition, obs_axs, chans_axs, time_axs, rng
            )
            if subject_labeled_array is None:
                continue  # Skip if subject has no data for this ROI

            # Concatenate subject's data into the ROI LabeledArray
            roi_labeled_array = concatenate_subject_labeled_arrays(
                roi_labeled_array, subject_labeled_array, chans_axs
            )

        # Add the concatenated LabeledArray to the ROI dictionary
        if roi_labeled_array is not None:
            roi_labeled_arrays[roi] = roi_labeled_array

    return roi_labeled_arrays

In [132]:
# Example usage:
roi_labeled_arrays = put_data_in_labeled_array_per_roi_subject(
    subjects_mne_objects,
    condition_names,
    rois,
    subjects,
    sig_electrodes_per_subject_roi,
    obs_axs=0,  # Trials dimension (ignoring the conditions dimension for now cuz we iterate over it)
    chans_axs=1,  # Channels dimension
    time_axs=2,   # Time dimension
    random_state=42  # For reproducibility
)

ROI 'lpfc': Maximum trials per condition:
  Condition 'Stimulus/BigLetters/SmallLetterh/Taskg': 59 trials from subjects ['D0057']
  Condition 'Stimulus/BigLetters/SmallLetterh/Taskl': 57 trials from subjects ['D0059']
  Condition 'Stimulus/BigLetters/SmallLetters/Taskg': 62 trials from subjects ['D0059']
  Condition 'Stimulus/BigLetters/SmallLetters/Taskl': 60 trials from subjects ['D0063']
  Condition 'Stimulus/BigLetterh/SmallLetterh/Taskg': 61 trials from subjects ['D0090']
  Condition 'Stimulus/BigLetterh/SmallLetterh/Taskl': 60 trials from subjects ['D0057']
  Condition 'Stimulus/BigLetterh/SmallLetters/Taskg': 59 trials from subjects ['D0071']
  Condition 'Stimulus/BigLetterh/SmallLetters/Taskl': 64 trials from subjects ['D0090']
in roi lpfc, subject D0057 has 59 trials for condition Stimulus/BigLetters/SmallLetterh/Taskg
in roi lpfc, subject D0057 has 54 trials for condition Stimulus/BigLetters/SmallLetterh/Taskl
in roi lpfc, subject D0057 has 53 trials for condition Stimulus/Bi

In [133]:
roi_labeled_arrays['lpfc'].shape

(8, 64, 59, 641)

remove trials with nans from roi labeled arrays if just getting the minimum number of trials for all subjects

In [134]:
def remove_nans_from_labeled_array(labeled_array, obs_axs=0, chans_axs=1, time_axs=2):
    """
    Remove trials that have NaN values from a LabeledArray and identify conditions with no valid trials.

    Parameters:
    - labeled_array: A LabeledArray with conditions, trials, channels, and timepoints dimensions.
    - obs_axs: The trials dimension 
    - chans_axs: The channels dimension
    - time_axs: The time dimension

    Returns:
    - labeled_array_no_nans: A LabeledArray with only trials that have no NaN values.
    - conditions_with_no_valid_trials: List of condition names with no valid trials after NaN removal.
    """

    # Initialize a dictionary to store data without NaNs for each condition
    reshaped_data_dict = {}
    # Initialize a list to keep track of conditions with no valid trials
    conditions_with_no_valid_trials = []

    # Extract the condition labels (which are in labeled_array.labels[0])
    condition_names = labeled_array.labels[0]  # Ensure we get the condition names

    # Loop over each condition
    for condition_name in condition_names:
        # Extract the data for the current condition
        condition_data = labeled_array[condition_name]  # Shape: (Trials, Channels, Timepoints)

        # Find the indices of trials that do not contain NaNs
        # Reduce over channels and time axes to check if any NaN exists in a trial
        valid_trial_indices = ~np.isnan(condition_data).any(axis=(chans_axs, time_axs))

        # valid_trial_indices is a boolean array of shape (Trials,)
        # Select only the valid trials
        condition_data_clean = condition_data[valid_trial_indices, :, :]

        # Check if there are valid trials
        if condition_data_clean.shape[obs_axs] > 0:
            # Store the processed data for this condition
            reshaped_data_dict[condition_name] = condition_data_clean
        else:
            print(f"No valid trials for condition '{condition_name}' after removing NaNs.")
            conditions_with_no_valid_trials.append(condition_name)

    # Optionally, print the conditions with no valid trials
    if conditions_with_no_valid_trials:
        print("Conditions with no valid trials after NaN removal:", conditions_with_no_valid_trials)

    # Proceed to create the LabeledArray if there are any valid conditions
    if len(reshaped_data_dict) == 0:
        raise ValueError("All conditions have no valid trials after removing NaNs.")

    # Create a new LabeledArray from the reshaped data dictionary
    labeled_array_no_nans = LabeledArray.from_dict(reshaped_data_dict)

    return labeled_array_no_nans, conditions_with_no_valid_trials


def remove_nans_from_all_roi_labeled_arrays(roi_labeled_arrays, obs_axs=0, chans_axs=1, time_axs=2):
    """
    Loop through all ROIs and apply the NaN removal function to each LabeledArray.

    Parameters:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.
    - obs_axs: The trials dimension.
    - chans_axs: The channels dimension.
    - time_axs: The time dimension.

    Returns:
    - roi_labeled_arrays_no_nans: Dictionary where keys are ROIs and values are LabeledArrays with NaNs removed.
    - conditions_with_no_valid_trials_per_roi: Dictionary where keys are ROIs and values are lists of condition names with no valid trials.
    """
    roi_labeled_arrays_no_nans = {}
    conditions_with_no_valid_trials_per_roi = {}

    # Loop through each ROI in the dictionary
    for roi, labeled_array in roi_labeled_arrays.items():
        # Apply the NaN removal function to the current labeled array
        labeled_array_no_nans, conditions_with_no_valid_trials = remove_nans_from_labeled_array(
            labeled_array, obs_axs=obs_axs, chans_axs=chans_axs, time_axs=time_axs
        )

        # Store the reshaped data for this ROI
        roi_labeled_arrays_no_nans[roi] = labeled_array_no_nans

        # Store the conditions with no valid trials for this ROI
        if conditions_with_no_valid_trials:
            conditions_with_no_valid_trials_per_roi[roi] = conditions_with_no_valid_trials

    return roi_labeled_arrays_no_nans, conditions_with_no_valid_trials_per_roi

In [135]:
roi_labeled_arrays_no_nans, conditions_with_no_valid_trials_per_roi = remove_nans_from_all_roi_labeled_arrays(roi_labeled_arrays, obs_axs=0, chans_axs=1, time_axs=2)

In [136]:
'''
11/6 need to update the obs_axs here to work with obs_axs=1 and also all the way down too 
(though I'm not sure why the obs_axs is 0 for decoding, maybe cuz I make arrays for each condition?)
'''
def concatenate_conditions_by_string(roi_labeled_arrays, roi, strings_to_find, obs_axs=0):
    """
    Concatenate trials across condition names that contain specific strings.
    Assign labels based on the groupings of the conditions.

    Parameters:
    - roi_labeled_arrays: Dictionary of LabeledArrays for each ROI.
    - roi: The specific ROI to process.
    - strings_to_find: List of strings or list of lists of strings to search for in condition names.
                      If a list of strings is provided, each string is treated as its own condition group.
                      If a list of lists is provided, each sublist represents a group of conditions.
    - obs_axs (int) : The trials dimension. Concatenation will happen along this axis. This is the 1st dimension (not 0th) because conditions in the labeled array is the 0th. But we will subtract 1 if not considering conditions as a dimension (looping over conditions)
    
    Returns:
    - concatenated_data: The concatenated trials by (channels, timepoints, or whatever your other dimensions are) across the matching conditions.
    - labels: A numpy array of labels (0, 1, 2, ...) corresponding to each group of conditions.
    - cats: Dictionary of {condition_name: index} for decoding.
    """
    concatenated_data = []
    labels = []
    cats = {}

    # Track current label index
    current_label = 0

    # Normalize strings_to_find so each entry is a list (whether it's a string or a list of strings)
    if isinstance(strings_to_find, list) and all(isinstance(s, str) for s in strings_to_find):
        # If it's a flat list of strings, convert each string into its own single-item list
        strings_to_find = [[s] for s in strings_to_find]

    # Iterate over each group (whether it's a single string or a list of strings)
    for string_group in strings_to_find:
        # Find condition names that match any of the strings in the current string_group
        matching_conditions = [cond for cond in roi_labeled_arrays[roi].keys() if any(s in cond for s in string_group)]

        if not matching_conditions:
            continue

        # Concatenate data for all matching conditions
        data_to_concatenate = []
        for cond in matching_conditions:
            # Extract data for the current condition
            data = roi_labeled_arrays[roi][cond]  # Shape: (trials, channels, timepoints)
            data_to_concatenate.append(data)
            
            # Update labels for the current condition group
            labels.extend([current_label] * data.shape[0])
        
        # Check if we have data to concatenate for this condition group
        if data_to_concatenate:
            concatenated_data.append(np.concatenate(data_to_concatenate, axis=obs_axs))

        # Assign current label to the condition group (based on the first string in the group for reference)
        cats[tuple(string_group)] = current_label
        current_label += 1

    # Ensure there is data to concatenate
    if not concatenated_data:
        raise ValueError(f"No matching conditions found for ROI: {roi} and strings: {strings_to_find}")

    # Concatenate all condition data along the trials axis
    concatenated_data = np.concatenate(concatenated_data, axis=obs_axs)
    
    return concatenated_data, np.array(labels), cats

In [137]:
def process_and_balance_data_for_decoding(
    roi_labeled_arrays, roi, strings_to_find, obs_axs, balance_method, random_state
):
    """
    Processes and balances the data for a given ROI.

    Parameters:
    - roi_labeled_arrays: Dictionary containing reshaped data for each ROI.
    - roi: The ROI to process.
    - strings_to_find: List of strings or string groups to identify condition labels.
    - obs_axs: The trials axis.
    - balance_method: 'pad_with_nans' or 'subsample' to balance trial counts between conditions.
    - random_state: Random seed for reproducibility.

    Returns:
    - concatenated_data: The processed and balanced numpy array for decoding. This gets the data out of the roi labeled arrays format.
    - labels: The processed labels array.
    - cats: Dictionary of condition categories.
    """
    rng = np.random.RandomState(random_state)

    # Concatenate the trials and get labels
    concatenated_data, labels, cats = concatenate_conditions_by_string(
        roi_labeled_arrays, roi, strings_to_find, obs_axs
    )

    print(f"Concatenated data shape for {roi}: {concatenated_data.shape}")

    if balance_method == 'subsample':
        # Remove NaN trials from concatenated_data and labels
        nan_trials = np.isnan(concatenated_data).any(axis=tuple(range(1, concatenated_data.ndim)))
        valid_trials = ~nan_trials

        # Keep only valid trials
        concatenated_data = concatenated_data[valid_trials]
        labels = labels[valid_trials]

    # Calculate trial counts per condition
    trial_counts = {}
    condition_indices = {}

    for string_group in strings_to_find:
        condition_label = cats[tuple(string_group) if isinstance(string_group, list) else (string_group,)]
        condition_trials = labels == condition_label
        data_for_condition = concatenated_data[condition_trials]

        # Store indices and counts
        condition_indices[condition_label] = np.where(condition_trials)[0]
        trial_counts[condition_label] = data_for_condition.shape[0]

        print(f'Condition {string_group} has {trial_counts[condition_label]} trials')

    if balance_method == 'pad_with_nans':
        max_trial_count = max(trial_counts.values())
        for condition_label, count in trial_counts.items():
            trials_to_add = max_trial_count - count
            if trials_to_add > 0:
                nan_trial_shape = (trials_to_add,) + concatenated_data.shape[1:]
                nan_trials = np.full(nan_trial_shape, np.nan)
                concatenated_data = np.concatenate([concatenated_data, nan_trials], axis=obs_axs)
                labels = np.concatenate([labels, [condition_label] * trials_to_add])
    elif balance_method == 'subsample':
        min_trial_count = min(trial_counts.values())
        subsampled_indices = []
        for condition_label in trial_counts.keys():
            indices = condition_indices[condition_label]
            if trial_counts[condition_label] > min_trial_count:
                selected_indices = rng.choice(indices, size=min_trial_count, replace=False)
            else:
                selected_indices = indices
            subsampled_indices.extend(selected_indices)

        subsampled_indices = np.array(subsampled_indices)
        concatenated_data = concatenated_data[subsampled_indices]
        labels = labels[subsampled_indices]
    else:
        raise ValueError(f"Invalid balance_method: {balance_method}. Choose 'pad_with_nans' or 'subsample'.")

    return concatenated_data, labels, cats

set up decoding output directory and conditions to compare 9/12

In [228]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")
save_dir = os.path.join(LAB_root, 'BIDS-1.1_GlobalLocal', 'BIDS', 'derivatives', 'decoding', 'figs')

if conditions == stimulus_experiment_conditions:
    condition_comparisons = {}
    condition_comparisons['congruency'] = [['c25.0', 'c75.0'], ['i25.0', 'i75.0']]
    condition_comparisons['switchType'] = [['r25.0', 'r75.0'], ['s25.0', 's75.0']]
    
elif conditions == stimulus_conditions:
    condition_comparisons = {}
    condition_comparisons['BigLetter'] = ['BigLetters', 'BigLetterh']
    condition_comparisons['SmallLetter'] = ['SmallLetters', 'SmallLetterh']
    condition_comparisons['Task'] = ['Taskg', 'Taskl']
    
    # strings_to_find = ['BigLetters', 'BigLetterh']
    # condition_0_name = 'BigLetters'
    # condition_1_name = 'BigLetterh'

    # strings_to_find = ['SmallLetters', 'SmallLetterh']
    # condition_0_name = 'SmallLetters'
    # condition_1_name = 'SmallLetterh'

    # strings_to_find = ['Taskg', 'Taskl']
    # condition_0_name = 'Taskg'
    # condition_1_name = 'Taskl'

In [139]:
def get_and_plot_confusion_matrix_for_rois_jim(
    roi_labeled_arrays, rois, condition_comparison, strings_to_find, save_dir,
    time_interval_name=None, n_splits=5, n_repeats=5, obs_axs=0, balance_method='pad_with_nans', random_state=42
):
    """
    Compute the confusion matrix for each ROI and return it. This function allows for balancing trial counts
    either by padding with NaNs or by subsampling trials to match the condition with the fewest valid (non-NaN) trials.
    
    Parameters:
    - roi_labeled_arrays: Dictionary containing the reshaped data for each ROI.
    - rois: List of regions of interest (ROIs) to process.
    - condition_comparison: The condition that we're comparing labels for (e.g., 'BigLetter').
    - strings_to_find: List of strings or string groups to identify condition labels.
    - save_dir: Directory to save the confusion matrix plots.
    - time_interval_name: Optional string to add to the filename for the time interval.
    - n_splits: Number of splits for cross-validation.
    - n_repeats: Number of repeats for cross-validation.
    - obs_axs: The trials axis.
    - balance_method: 'pad_with_nans' or 'subsample' to balance trial counts between conditions.
    - random_state: Random seed for reproducibility.
    
    Returns:
    - confusion_matrices: Dictionary containing confusion matrices for each ROI.
    """
    confusion_matrices = {}
    rng = np.random.RandomState(random_state)

    for roi in rois:
        print(f"Processing ROI: {roi}")
        concatenated_data, labels, cats = process_and_balance_data_for_decoding(
            roi_labeled_arrays, roi, strings_to_find, obs_axs, balance_method, random_state
        )

        # Create a Decoder and run cross-validation
        decoder = Decoder(cats, 0.80, oversample=True, n_splits=n_splits, n_repeats=n_repeats)

        # Use the concatenated data for the decoder
        cm = decoder.cv_cm_jim(concatenated_data, labels, normalize='true', obs_axs=obs_axs)
        cm_avg = np.mean(cm, axis=0)

        # Store the confusion matrix in the dictionary
        confusion_matrices[roi] = cm_avg

        # Convert tuple labels to simple strings for display
        display_labels = [
            key[0] if isinstance(key, tuple) and len(key) == 1 else str(key)
        for key in cats.keys()
        ]
        
        # Plot the Confusion Matrix
        fig, ax = plt.subplots()
        disp = ConfusionMatrixDisplay(confusion_matrix=cm_avg, display_labels=display_labels)
        disp.plot(ax=ax)

        # Save the figure with the time interval in the filename
        time_str = f"_{time_interval_name}" if time_interval_name else ""
        file_name = (
            f'{roi}_{condition_comparison}{time_str}_time_averaged_confusion_matrix_'
            f'{n_splits}splits_{n_repeats}repeats_{balance_method}.png'
        )
        # plt.savefig(os.path.join(save_dir, file_name))
        plt.close()

    return confusion_matrices


In [147]:
# get the confusion matrix using the downsampled version
for condition_comparison, strings_to_find in condition_comparisons.items():
    confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
        roi_labeled_arrays=roi_labeled_arrays_no_nans,
        rois=rois,
        condition_comparison=condition_comparison,
        strings_to_find=strings_to_find,
        save_dir=save_dir,
        time_interval_name=None,
        n_splits=5,
        n_repeats=10,
        obs_axs=0,
        balance_method='subsample',  # Use 'subsample' to balance by subsampling
        random_state=42  # For reproducibility
    )

Processing ROI: lpfc
Concatenated data shape for lpfc: (272, 59, 641)
Condition BigLetters has 83 trials
Condition BigLetterh has 100 trials
Processing ROI: v1
Concatenated data shape for v1: (344, 11, 641)
Condition BigLetters has 158 trials
Condition BigLetterh has 166 trials
Processing ROI: occ
Concatenated data shape for occ: (288, 51, 641)
Condition BigLetters has 96 trials
Condition BigLetterh has 110 trials
Processing ROI: occ_filtered
Concatenated data shape for occ_filtered: (344, 23, 641)
Condition BigLetters has 119 trials
Condition BigLetterh has 128 trials
Processing ROI: occ_best_filtered
Concatenated data shape for occ_best_filtered: (392, 10, 641)
Condition BigLetters has 139 trials
Condition BigLetterh has 138 trials
Processing ROI: lpfc
Concatenated data shape for lpfc: (272, 59, 641)
Condition SmallLetters has 88 trials
Condition SmallLetterh has 95 trials
Processing ROI: v1
Concatenated data shape for v1: (344, 11, 641)
Condition SmallLetters has 164 trials
Conditio

In [24]:
# get the confusion matrix using the version with nan trials and doing nan trial mixup
for condition_comparison, strings_to_find in condition_comparisons.items():
    confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
        roi_labeled_arrays=roi_labeled_arrays,
        rois=rois,
        condition_comparison=condition_comparison,
        strings_to_find=strings_to_find,
        save_dir=save_dir,
        time_interval_name=None,
        n_splits=5,
        n_repeats=10,
        obs_axs=0,
        balance_method='pad_with_nans',  # Use 'pad_with_nans' to balance by padding with nans
        random_state=42  # For reproducibility
    )


Processing ROI: lpfc
Concatenated data shape for lpfc: (1024, 59, 641)
Condition ['c25.0', 'c75.0'] has 512 trials
Condition ['i25.0', 'i75.0'] has 512 trials
Processing ROI: v1
Concatenated data shape for v1: (1008, 11, 641)
Condition ['c25.0', 'c75.0'] has 504 trials
Condition ['i25.0', 'i75.0'] has 504 trials
Processing ROI: occ
Concatenated data shape for occ: (1008, 51, 641)
Condition ['c25.0', 'c75.0'] has 504 trials
Condition ['i25.0', 'i75.0'] has 504 trials
Processing ROI: occ_filtered
Concatenated data shape for occ_filtered: (1008, 23, 641)
Condition ['c25.0', 'c75.0'] has 504 trials
Condition ['i25.0', 'i75.0'] has 504 trials
Processing ROI: occ_best_filtered
Concatenated data shape for occ_best_filtered: (1008, 10, 641)
Condition ['c25.0', 'c75.0'] has 504 trials
Condition ['i25.0', 'i75.0'] has 504 trials
Processing ROI: lpfc
Concatenated data shape for lpfc: (1024, 59, 641)
Condition ['r25.0', 'r75.0'] has 512 trials
Condition ['s25.0', 's75.0'] has 512 trials
Processing

prestimulus vs poststimulus confusion matrices 9/21

In [144]:
def get_data_in_time_range(labeled_array, time_range, time_axs=-1):
    """
    Extract data from a LabeledArray where the time points fall within a given range, using the LabeledArray `take` function.
    
    Parameters:
    - labeled_array: The LabeledArray containing time points as labels.
    - time_range: A tuple (start_time, end_time) representing the range of time.
    - time_axs: The time dimension.
    
    Returns:
    - filtered_data: A LabeledArray containing only the data where the timepoints are within the specified range.
    """
    start_time, end_time = time_range

    # Assume that the time labels are stored in the last dimension, but can change this.
    time_points = np.array(labeled_array.labels[time_axs], dtype=float)  # convert to floats, ensure conversion to float numpy array

    # Find the indices of time points within the specified range
    time_indices = np.where((time_points >= start_time) & (time_points <= end_time))[0]

    # Use the take function to select the time indices along the time axis (axis=3)
    filtered_data = labeled_array.take(time_indices, axis=3)

    return filtered_data

In [145]:
# # Define the time intervals of interest
# pre_stimulus_interval = (-1.0, 0.0)  # Pre-stimulus: time -1 to 0 seconds
# post_stimulus_interval = (0.0, 1.5)  # Post-stimulus: time 0 to 1.5 seconds
# pre_stimulus_roi_labeled_arrays = {}
# post_stimulus_roi_labeled_arrays = {}

# for roi, labeled_array in roi_labeled_arrays.items():
#     pre_stimulus_roi_labeled_arrays[roi] = get_data_in_time_range(labeled_array, pre_stimulus_interval)
#     post_stimulus_roi_labeled_arrays[roi] = get_data_in_time_range(labeled_array, post_stimulus_interval)


# pre_stimulus_confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
#     roi_labeled_arrays=pre_stimulus_roi_labeled_arrays,
#     rois=rois,
#     condition_comparison=condition_comparison,
#     strings_to_find=strings_to_find,
#     save_dir=save_dir,
#     time_interval_name='pre_stimulus',  # If not dealing with specific time intervals
#     n_splits=5,
#     n_repeats=10,
#     obs_axs=0,
#     balance_method='subsample',  # Use 'subsample' to balance by subsampling
#     random_state=42  # For reproducibility
# )

# post_stimulus_confusion_matrices = get_and_plot_confusion_matrix_for_rois_jim(
#     roi_labeled_arrays=post_stimulus_roi_labeled_arrays,
#     rois=rois,
#     condition_comparison=condition_comparison,
#     strings_to_find=strings_to_find,
#     save_dir=save_dir,
#     time_interval_name='post_stimulus',  # If not dealing with specific time intervals
#     n_splits=5,
#     n_repeats=10,
#     obs_axs=0,
#     balance_method='subsample',  # Use 'subsample' to balance by subsampling
#     random_state=42  # For reproducibility
# )

Processing ROI: lpfc


ValueError: No matching conditions found for ROI: lpfc and strings: [['c25.0', 'c75.0'], ['i25.0', 'i75.0']]

define colors for plotting (not used yet as of 8/21)

In [32]:
# Define colors for the model names
colors = {
    'congruency': 'red',
    'switchType': 'blue',
    'congruencyProportion': 'pink',
    'switchProportion': 'skyblue',
    'congruency_congruencyProportion': 'hotpink',
    'congruency_congruency_proportion': 'hotpink',
    'switchType_switchProportion': 'gray',
    'switch_type_switch_proportion': 'gray',
    'bigLetter': 'green',
    'big_letter': 'green',
    'smallLetter': 'orange',
    'small_letter': 'orange',
    'task': 'gray',
    'c75.0': 'pink',
    'i75.0': 'pink',
    'c25.0': 'gold',
    'i25.0': 'gold',
    'r25.0': 'lightblue',
    's25.0': 'lightblue',
    'r75.0': 'purple',
    's75.0': 'purple'
}

# Define linestyles for the model names
linestyles = {
    'big letter S': '-',
    'BigLetters': '-',

    'big letter H': '--',
    'BigLetterh': '--',

    'small letter S': '-',
    'SmallLetters': '-',

    'small letter H': '--',
    'SmallLetterh': '--',

    'task G': '-',
    'Taskg': '-',

    'task L': '--',
    'Taskl': '--',

    'congruent': '-',
    'c': '-',

    'incongruent': '--',
    'i': '--',

    'repeat': '-',
    'r': '-',

    'switch': '--',
    's': '--',

    'c25.0': '-',
    'c75.0': '-',
    'i25.0': '--',
    'i75.0': '--'
}

new attempt at time window sliding decoding with significance 11/23  
TODO: Clean this up.   
Make subfunctions to break this down. Everything before defining the Decoder objects can be a function, that can be shared between this and the whole time window version.   
The decoder true and decoder shuffle can be done with a function maybe.   
And maybe return just accuracies, which I can then call this entire function separately for true and shuffled.  

  
ALSO STORE THE SHUFFLED OUTPUT IN A NUMPY ARRAY SO I DON'T HAVE TO MAKE IT EVERY TIME

In [244]:
# copy this back into jim_decoding_functions.py once it's working! 12/6
import numpy as np
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from ieeg.decoding.decoders import PcaLdaClassification
from ieeg.calc.oversample import MinimumNaNSplit
from ieeg.calc.fast import mixup
from scipy.stats import norm
from tqdm import tqdm
from numpy.lib.stride_tricks import as_strided
import itertools
from joblib import Parallel, delayed
from numpy.lib.stride_tricks import sliding_window_view

# largely stolen from aaron's ieeg plot_decoding.py

def mixup2(arr: np.ndarray, labels: np.ndarray, obs_axs: int, alpha: float = 1.,
          seed: int = None) -> None:
    """Mixup the data using the labels

    Parameters
    ----------
    arr : array
        The data to mixup.
    labels : array
        The labels to use for mixing.
    obs_axs : int
        The axis along which to apply func.
    alpha : float
        The alpha value for the beta distribution.
    seed : int
        The seed for the random number generator.

    Examples
    --------
    >>> np.random.seed(0)
    >>> arr = np.array([[1, 2], [4, 5], [7, 8],
    ... [float("nan"), float("nan")]])
    >>> labels = np.array([0, 0, 1, 1])
    >>> mixup2(arr, labels, 0)
    >>> arr
    array([[1.        , 2.        ],
           [4.        , 5.        ],
           [7.        , 8.        ],
           [6.03943491, 7.03943491]])
           """
    if arr.ndim > 2:
        arr = arr.swapaxes(obs_axs, -2)
        for i in range(arr.shape[0]):
            mixup2(arr=arr[i], labels=labels, obs_axs=obs_axs, alpha=alpha, seed=seed)
    else:
        if seed is not None:
            np.random.seed(seed)
        if obs_axs == 1:
            arr = arr.T

        n_nan = np.where(np.isnan(arr).any(axis=1))[0]
        n_non_nan = np.where(~np.isnan(arr).any(axis=1))[0]

        for i in n_nan:
            l_class = labels[i]
            possible_choices = np.nonzero(np.logical_and(~np.isnan(arr).any(axis=1), labels == l_class))[0]
            choice1 = np.random.choice(possible_choices)
            choice2 = np.random.choice(n_non_nan)
            l = np.random.beta(alpha, alpha)
            if l < .5:
                l = 1 - l
            arr[i] = l * arr[choice1] + (1 - l) * arr[choice2]

class Decoder(PcaLdaClassification, MinimumNaNSplit):
    def __init__(self, categories: dict, *args, n_splits: int = 5, n_repeats: int = 10,
                 oversample: bool = True, max_features: int = float("inf"), **kwargs):
        PcaLdaClassification.__init__(self, *args, **kwargs)
        MinimumNaNSplit.__init__(self, n_splits, n_repeats)
        if not oversample:
            self.oversample = lambda x, func, axis: x
        self.categories = categories
        self.max_features = max_features

    def cv_cm_jim(self, x_data: np.ndarray, labels: np.ndarray,
              normalize: str = None, obs_axs: int = -2):
        n_cats = len(set(labels))
        mats = np.zeros((self.n_repeats, self.n_splits, n_cats, n_cats))
        obs_axs = x_data.ndim + obs_axs if obs_axs < 0 else obs_axs
        idx = [slice(None) for _ in range(x_data.ndim)]
        for f, (train_idx, test_idx) in enumerate(self.split(x_data.swapaxes(0, obs_axs), labels)):
            x_train = np.take(x_data, train_idx, obs_axs)
            x_test = np.take(x_data, test_idx, obs_axs)
            
            y_train = labels[train_idx]
            mixup2(arr=x_train, labels=y_train, obs_axs=obs_axs, alpha=1., seed=None)
            y_test = labels[test_idx]
            # for i in set(labels):
            #     # fill in train data nans with random combinations of existing train data trials (mixup)
            #     idx[obs_axs] = y_train == i
            #     x_train[tuple(idx)] = self.oversample(x_train[tuple(idx)], axis=obs_axs, func=mixup)

            # fill in test data nans with noise from distribution
            is_nan = np.isnan(x_test)
            x_test[is_nan] = np.random.normal(0, 1, np.sum(is_nan))

            # feature selection
            train_in = flatten_features(x_train, obs_axs)
            test_in = flatten_features(x_test, obs_axs)
            if train_in.shape[1] > self.max_features:
                tidx = np.random.choice(train_in.shape[1], self.max_features, replace=False)
                train_in = train_in[:, tidx]
                test_in = test_in[:, tidx]

            # fit model and score results
            self.fit(train_in, y_train)
            pred = self.predict(test_in)
            rep, fold = divmod(f, self.n_splits)
            mats[rep, fold] = confusion_matrix(y_test, pred)

        # average the repetitions, sum the folds
        matk = np.sum(mats, axis=1)
        if normalize == 'true':
            divisor = np.sum(matk, axis=-1, keepdims=True)
        elif normalize == 'pred':
            divisor = np.sum(matk, axis=-2, keepdims=True)
        elif normalize == 'all':
            divisor = self.n_repeats
        else:
            divisor = 1
        return matk / divisor
    
    # untested 11/30
    def cv_cm_jim_window_shuffle(self, x_data: np.ndarray, labels: np.ndarray,
                normalize: str = None, obs_axs: int = -2, time_axs: int = -1, n_jobs: int = 1,
                window: int = None, step_size: int = 1,
                    shuffle: bool = False, oversample: bool = True) -> np.ndarray:
        """Cross-validated confusion matrix with windowing and optional shuffling."""
        n_cats = len(set(labels))
        time_axs_positive = time_axs % x_data.ndim
        out_shape = (self.n_repeats, self.n_splits, n_cats, n_cats)

        if window is not None:
            # Include the step size in the windowed output shape
            steps = (x_data.shape[time_axs_positive] - window) // step_size + 1
            out_shape = (steps,) + out_shape
                
        mats = np.zeros(out_shape, dtype=np.uint8)
        data = x_data.swapaxes(0, obs_axs)

        if shuffle:
            # shuffled label pool
            label_stack = []
            for i in range(self.n_repeats):
                label_stack.append(labels.copy())
                self.shuffle_labels(data, label_stack[-1], 0)

            # build the test/train indices from the shuffled labels for each
            # repetition, then chain together the repetitions
            # splits = (train, test)

            print("Shuffle validation:")
            for i, labels in enumerate(label_stack):
                # Compare with the first repetition to ensure variety in shuffles
                if i > 0:
                    diff = np.sum(label_stack[0] != labels)

            idxs = ((self.split(data, l), l) for l in label_stack)
            idxs = ((itertools.islice(s, self.n_splits),
                     itertools.repeat(l, self.n_splits))
                    for s, l in idxs)
            splits, label = zip(*idxs)
            splits = itertools.chain.from_iterable(splits)
            label = itertools.chain.from_iterable(label)
            idxs = zip(splits, label)

        else:
            idxs = ((splits, labels) for splits in self.split(data, labels))

        # 11/1 below is aaron's code for windowing. 
        def proc(train_idx, test_idx, l):
            x_stacked, y_train, y_test = sample_fold(train_idx, test_idx, data, l, 0, oversample)
            print(f"x_stacked shape: {x_stacked.shape}")

            # Use the updated windower function with step_size
            windowed = windower(x_stacked, window, axis=time_axs, step_size=step_size)
            print(f"windowed shape: {windowed.shape}")

            out = np.zeros((windowed.shape[0], n_cats, n_cats), dtype=np.uint8)
            for i, x_window in enumerate(windowed):
                x_flat = x_window.reshape(x_window.shape[0], -1)
                x_train, x_test = np.split(x_flat, [train_idx.shape[0]], 0)
                out[i] = self.fit_predict(x_train, x_test, y_train, y_test)
            return out

        # # loop over folds and repetitions
        if n_jobs == 1:
            idxs = tqdm(idxs, total=self.n_splits * self.n_repeats)
            results = (proc(train_idx, test_idx, l) for (train_idx, test_idx), l in idxs)
        else:
            results = Parallel(n_jobs=n_jobs, return_as='generator', verbose=40)(
                delayed(proc)(train_idx, test_idx, l)
                for (train_idx, test_idx), l in idxs)

        # # Collect the results
        for i, result in enumerate(results):
            rep, fold = divmod(i, self.n_splits)
            mats[:, rep, fold] = result

        # normalize, sum the folds
        mats = np.sum(mats, axis=-3)
        if normalize == 'true':
            divisor = np.sum(mats, axis=-1, keepdims=True)
        elif normalize == 'pred':
            divisor = np.sum(mats, axis=-2, keepdims=True)
        elif normalize == 'all':
            divisor = self.n_repeats
        else:
            divisor = 1
        return mats / divisor
        
    def fit_predict(self, x_train, x_test, y_train, y_test):
        # fit model and score results
        self.model.fit(x_train, y_train)
        pred = self.model.predict(x_test)
        return confusion_matrix(y_test, pred)
    
    def cv_cm_return_scores(self, x_data: np.ndarray, labels: np.ndarray,
                            normalize: str = None, obs_axs: int = -2):
        '''
        trying to get the scores manually from cv cm but i realize that in decoders.py, PcaLdaClassification already has a get_scores function. Try get_scores with shuffle=True to get fake, permuted scores.
        '''
        # Get the confusion matrix by calling `cv_cm`
        cm = self.cv_cm_jim(x_data, labels, normalize, obs_axs)

        # Average the confusion matrices across the repetitions
        cm_avg = np.mean(cm, axis=0)  # Now cm_avg will be of shape (2, 2)

        # Calculate the individual decoding scores (Accuracy, Precision, etc.)
        scores = self.calculate_scores(cm_avg)

        return cm_avg, scores

    def calculate_scores(self, cm):
        """
        Calculate the individual decoding scores from the confusion matrix. 10/27 Ugh Aaron already does this directly in the PcaLdaClassification class... 

        Parameters:
        - cm: The confusion matrix (averaged over folds).

        Returns:
        - scores: A dictionary containing the scores (accuracy, precision, recall, f1, d-prime) for each class.
        """
        scores = {}
        tp = np.diag(cm)  # True Positives
        fp = np.sum(cm, axis=0) - tp  # False Positives
        fn = np.sum(cm, axis=1) - tp  # False Negatives
        tn = np.sum(cm) - (fp + fn + tp)  # True Negatives

        # Calculate accuracy, precision, recall, and f1 score
        accuracy = np.sum(tp) / np.sum(cm)
        precision = tp / (tp + fp + 1e-8)
        recall = tp / (tp + fn + 1e-8)
        f1 = 2 * (precision * recall) / (precision + recall + 1e-8)

        # Store the basic scores
        scores['accuracy'] = accuracy
        scores['precision'] = precision
        scores['recall'] = recall
        scores['f1'] = f1

        # Calculate hit rate and false alarm rate
        hit_rate = recall  # Hit rate is the same as recall (TP / (TP + FN))
        false_alarm_rate = fp / (fp + tn + 1e-8)  # False alarm rate (FP / (FP + TN))

        # Ensure hit_rate and false_alarm_rate are in valid range [0, 1] for Z-transform
        hit_rate = np.clip(hit_rate, 1e-8, 1 - 1e-8)
        false_alarm_rate = np.clip(false_alarm_rate, 1e-8, 1 - 1e-8)

        # Z-transform to calculate d-prime
        z_hit_rate = norm.ppf(hit_rate)  # Z-transform for hit rate
        z_false_alarm_rate = norm.ppf(false_alarm_rate)  # Z-transform for false alarm rate

        # Calculate d-prime
        d_prime = z_hit_rate - z_false_alarm_rate

        # Store d-prime in the scores dictionary
        scores['d_prime'] = d_prime

        return scores

    
def flatten_features(arr: np.ndarray, obs_axs: int = -2) -> np.ndarray:
    obs_axs = arr.ndim + obs_axs if obs_axs < 0 else obs_axs
    if obs_axs != 0:
        out = arr.swapaxes(0, obs_axs)
    else:
        out = arr.copy()
    return out.reshape(out.shape[0], -1)

def windower(x_data: np.ndarray, window_size: int = None, axis: int = -1,
             step_size: int = 1, insert_at: int = 0):
    if window_size is None:
        return x_data[np.newaxis, ...]  # Add a new axis for compatibility

    axis = axis % x_data.ndim
    data_length = x_data.shape[axis]
    
    # Compute the number of full steps (exclude remainder)
    full_steps = (data_length - window_size) // step_size + 1

    # Create the sliding window view for full windows
    windowed = sliding_window_view(x_data, window_shape=window_size, axis=axis)
    
    # Apply step_size by slicing
    if step_size > 1:
        slicing = [slice(None)] * windowed.ndim
        slicing[axis] = slice(0, None, step_size) # try 0, None, step_size for now, I think this should exclude the remainder..? Keep debugging with the bottom cell.
        windowed = windowed[tuple(slicing)]
    
    # Move the window dimension to the desired location
    if insert_at != axis:
        windowed = np.moveaxis(windowed, axis, insert_at)
    
    return windowed




# this is aaron's windower function. Replace with my windower that accounts for step size.
# def windower(x_data: np.ndarray, window_size: int, axis: int = -1, insert_at: int = 0):
#     """Create a sliding window view of the array with a given window size."""
#     # Compute the shape and strides for the sliding window view
#     shape = list(x_data.shape)
#     shape[axis] = x_data.shape[axis] - window_size + 1
#     shape.insert(axis, window_size)
#     strides = list(x_data.strides)
#     strides.insert(axis, x_data.strides[axis])

#     # Create the sliding window view
#     out = as_strided(x_data, shape=shape, strides=strides)

#     # Move the window size dimension to the front
#     out = np.moveaxis(out, axis, insert_at)

#     return out

# modified by jim 11/23, check aaron_decoding_init.py for original.
def sample_fold(train_idx: np.ndarray, test_idx: np.ndarray,
                x_data: np.ndarray, labels: np.ndarray,
                axis: int, oversample: bool = True):

    # Combine train and test indices
    idx_stacked = np.concatenate((train_idx, test_idx))
    x_stacked = np.take(x_data, idx_stacked, axis)
    y_stacked = labels[idx_stacked]

    # Split into training and testing sets
    sep = train_idx.shape[0]
    y_train, y_test = np.split(y_stacked, [sep])
    x_train, x_test = np.split(x_stacked, [sep], axis=axis)

    if oversample:
        # Apply mixup2 to x_train
        mixup2(arr=x_train, labels=y_train, obs_axs=axis, alpha=1., seed=None)

    # Fill in test data nans with noise from distribution
    is_nan = np.isnan(x_test)
    x_test[is_nan] = np.random.normal(0, 1, np.sum(is_nan))

    # Recombine x_train and x_test
    x_stacked = np.concatenate((x_train, x_test), axis=axis)

    return x_stacked, y_train, y_test


In [245]:
def get_confusion_matrices_for_rois_time_window_decoding_jim(
    roi_labeled_arrays, rois, condition_comparison, strings_to_find, save_dir,
    time_interval_name=None, n_splits=5, n_repeats=5, obs_axs=0, time_axs=-1,
    balance_method='pad_with_nans', random_state=42, window_size=None,
    step_size=1, n_permutations=100, sampling_rate=256, first_time_point=-1
):
    # Initialize dictionaries to store confusion matrices for each ROI
    cm_true_per_roi = {}
    cm_shuffle_per_roi = {}
    rng = np.random.RandomState(random_state)
    first_sample = first_time_point * sampling_rate

    for roi in rois:
        print(f"Processing ROI: {roi}")

        concatenated_data, labels, cats = process_and_balance_data_for_decoding(
            roi_labeled_arrays, roi, strings_to_find, obs_axs, balance_method, random_state
        )

        # Get the number of timepoints
        time_axis_length = concatenated_data.shape[time_axs]

        # Determine effective window size and step size
        if window_size is None:
            effective_window_size = time_axis_length
            effective_step_size = time_axis_length  # No overlap
            n_windows = 1
            start_times = [0] # only one window
        else:
            effective_window_size = window_size
            effective_step_size = step_size
            n_windows = (time_axis_length - effective_window_size) // effective_step_size + 1
            # Apply first_time_point offset
            start_times = [first_sample + effective_step_size * i for i in range(n_windows)]
            
        print(f"start times are: {start_times}")
        print(f"Effective window size: {effective_window_size}")
        print(f"Effective step size: {effective_step_size}")
        print(f"Number of windows: {n_windows}")

        # Calculate time centers based on window size and step size
        time_window_centers = [
            (start + effective_window_size / 2) / sampling_rate
            for start in start_times
        ]
        print(f"time_window_centers are: {time_window_centers}")
        
        # Create Decoder instances
        decoder_true = Decoder(cats, 0.80, oversample=True, n_splits=n_splits, n_repeats=n_repeats)
        decoder_shuffle = Decoder(cats, 0.80, oversample=True, n_splits=n_splits, n_repeats=n_permutations)

        # Run decoding with true labels
        cm_true = decoder_true.cv_cm_jim_window_shuffle(
            concatenated_data, labels, normalize='true', obs_axs=obs_axs, time_axs=time_axs,
            window=effective_window_size, step_size=effective_step_size, shuffle=False
        )

        # Run decoding with shuffled labels
        cm_shuffle = decoder_shuffle.cv_cm_jim_window_shuffle(
            concatenated_data, labels, normalize='true', obs_axs=obs_axs, time_axs=time_axs,
            window=effective_window_size, step_size=effective_step_size, shuffle=True
        )

        # Store the confusion matrices and time info
        cm_true_per_roi[roi] = {
            'cm_true': cm_true,  # Shape: (n_windows, n_repeats, n_classes, n_classes)
            'time_window_centers': time_window_centers,
            'window_size': effective_window_size,
            'step_size': effective_step_size,
            'condition_comparison': condition_comparison
        }

        cm_shuffle_per_roi[roi] = {
            'cm_shuffle': cm_shuffle,  # Shape: (n_windows, n_permutations, n_classes, n_classes)
            'time_window_centers': time_window_centers,
            'window_size': effective_window_size,
            'step_size': effective_step_size,
            'condition_comparison': condition_comparison
        }

    return cm_true_per_roi, cm_shuffle_per_roi

actually do the time window decoding, run stats, and plot accuracies

In [246]:
def compute_accuracies(cm_true, cm_shuffle):
    """
    Compute accuracies from confusion matrices.
    """
    n_windows = cm_true.shape[0]
    n_repeats = cm_true.shape[1]
    n_permutations = cm_shuffle.shape[1]

    accuracies_true = np.zeros((n_windows, n_repeats))
    accuracies_shuffle = np.zeros((n_windows, n_permutations))

    for win_idx in range(n_windows):
        # True accuracies
        for rep_idx in range(n_repeats):
            cm = cm_true[win_idx, rep_idx]
            accuracies_true[win_idx, rep_idx] = np.trace(cm) / np.sum(cm)
        # Shuffled accuracies
        for perm_idx in range(n_permutations):
            cm = cm_shuffle[win_idx, perm_idx]
            accuracies_shuffle[win_idx, perm_idx] = np.trace(cm) / np.sum(cm)

    return accuracies_true, accuracies_shuffle

def perform_time_perm_cluster_test(accuracies_true, accuracies_shuffle, p_thresh=0.05, n_perm=50, seed=42):
    """
    Perform time permutation cluster test.
    """
    accuracies_true_T = accuracies_true.T
    accuracies_shuffle_T = accuracies_shuffle.T

    significant_clusters, p_values = time_perm_cluster(
        accuracies_true_T,
        accuracies_shuffle_T,
        p_thresh=p_thresh,
        n_perm=n_perm,
        tails=1,
        axis=0,
        stat_func=lambda x, y, axis: np.mean(x, axis=axis),
        n_jobs=1,
        seed=seed
    )
    return significant_clusters, p_values

def plot_accuracies(time_points, accuracies_true, accuracies_shuffle, significant_clusters,
                    window_size, step_size, sampling_rate, condition_comparison, roi, save_dir):
    """
    Plot accuracies and save the plot with significant clusters marked by horizontal bars and asterisks.
    The time axis can start at a custom value specified by time_start.
    """
    n_repeats = accuracies_true.shape[1]
    n_permutations = accuracies_shuffle.shape[1]

    # Compute mean and standard error
    mean_true_accuracy = np.mean(accuracies_true, axis=1)
    se_true_accuracy = np.std(accuracies_true, axis=1) / np.sqrt(n_repeats)

    mean_shuffle_accuracy = np.mean(accuracies_shuffle, axis=1)
    se_shuffle_accuracy = np.std(accuracies_shuffle, axis=1) / np.sqrt(n_permutations)

    # Plotting
    plt.figure(figsize=(12, 6))
    plt.plot(time_points, mean_true_accuracy, label='True Accuracy', color='blue')
    plt.fill_between(
        time_points,
        mean_true_accuracy - se_true_accuracy,
        mean_true_accuracy + se_true_accuracy,
        alpha=0.2,
        color='blue'
    )

    plt.plot(time_points, mean_shuffle_accuracy, label='Shuffled Accuracy', color='red')
    plt.fill_between(
        time_points,
        mean_shuffle_accuracy - se_shuffle_accuracy,
        mean_shuffle_accuracy + se_shuffle_accuracy,
        alpha=0.2,
        color='red'
    )

    # Compute window duration
    window_duration = window_size / sampling_rate

    # Find contiguous significant clusters
    def find_clusters(significant_clusters):
        clusters = []
        in_cluster = False
        for idx, val in enumerate(significant_clusters):
            if val and not in_cluster:
                # Start of a new cluster
                start_idx = idx
                in_cluster = True
            elif not val and in_cluster:
                # End of the cluster
                end_idx = idx - 1
                clusters.append((start_idx, end_idx))
                in_cluster = False
        # Handle the case where the last value is in a cluster
        if in_cluster:
            end_idx = len(significant_clusters) - 1
            clusters.append((start_idx, end_idx))
        return clusters

    clusters = find_clusters(significant_clusters)

    # # Determine y position for the bars
    # max_y = np.max(mean_true_accuracy + se_true_accuracy)
    # min_y = np.min(mean_shuffle_accuracy - se_shuffle_accuracy)
    # y_bar = max_y + 0.02  # Adjust as needed
    # plt.ylim([min_y, y_bar + 0.05])  # Adjust ylim to accommodate the bars

    # Set y_bar to a fixed value within the y-axis limits
    y_bar = 0.95  # Fixed value near the top of the y-axis

    # Plot horizontal bars and asterisks for significant clusters
    for cluster in clusters:
        start_idx, end_idx = cluster
        start_time = time_points[start_idx] - (window_duration / 2)
        end_time = time_points[end_idx] + (window_duration / 2)
        plt.hlines(y=y_bar, xmin=start_time, xmax=end_time, color='black', linewidth=2)
        # Place an asterisk at the center of the bar
        center_time = (start_time + end_time) / 2
        plt.text(center_time, y_bar + 0.01, '*', ha='center', va='bottom', fontsize=14)

    # Set axis limits
    plt.ylim(0, 1)  # Y-axis limits
    plt.xlim(-1, 1.5)  # X-axis limits

    plt.xlabel('Time from Stim Onset (s)')
    plt.ylabel('Accuracy')
    plt.title(f'Decoding Accuracy over Time for {condition_comparison} in ROI {roi}')
    plt.legend()

    # Construct the filename
    filename = f"{condition_comparison}_ROI_{roi}_window{window_size}_step{step_size}.png"
    filepath = os.path.join(save_dir, filename)

    # Ensure save_dir exists
    os.makedirs(save_dir, exist_ok=True)

    # Save and close the plot
    plt.savefig(filepath)
    plt.close()

In [247]:
# Main code

# Directory to save confusion matrices
cm_save_dir = os.path.join(save_dir, "confusion_matrices")
os.makedirs(cm_save_dir, exist_ok=True)

condition_comparison_confusion_matrices = {}
n_splits=5
n_repeats=10
balance_method='subsample'
random_state=42
window_size=64
step_size=32
n_permutations=100

for condition_comparison, strings_to_find in condition_comparisons.items():
    # Get confusion matrices for each ROI
    cm_true_per_roi, cm_shuffle_per_roi = get_confusion_matrices_for_rois_time_window_decoding_jim(
        roi_labeled_arrays=roi_labeled_arrays,
        rois=rois,
        condition_comparison=condition_comparison,
        strings_to_find=strings_to_find,
        save_dir=save_dir,
        time_interval_name=None,
        n_splits=n_splits,
        n_repeats=n_repeats,
        obs_axs=0,
        time_axs=-1,
        balance_method=balance_method,
        random_state=random_state,
        window_size=window_size,
        step_size=step_size,
        n_permutations=n_permutations,
        sampling_rate=sampling_rate,
        first_time_point=-1
    )

    np.save(os.path.join(cm_save_dir, f'{condition_comparison}_{n_splits}_splits_{n_repeats}_repeats_{balance_method}_balance_method_{random_state}_random_state_{window_size}_window_size_{step_size}_step_size_{n_permutations}_permutations_{sampling_rate}_sampling_rate_cm_true_per_roi.npy'), cm_true_per_roi)
    np.save(os.path.join(cm_save_dir, f'{condition_comparison}_{n_splits}_splits_{n_repeats}_repeats_{balance_method}_balance_method_{random_state}_random_state_{window_size}_window_size_{step_size}_step_size_{n_permutations}_permutations_{sampling_rate}_sampling_rate_cm_shuffle_per_roi.npy'), cm_shuffle_per_roi)

    # Store the results in a dictionary
    condition_comparison_confusion_matrices[condition_comparison] = {
        'strings_to_find': strings_to_find,
        'cm_true_per_roi': cm_true_per_roi,
        'cm_shuffle_per_roi': cm_shuffle_per_roi
    }

    # Now compute accuracies and perform time permutation cluster test
    for roi in rois:
        cm_true = cm_true_per_roi[roi]['cm_true']
        cm_shuffle = cm_shuffle_per_roi[roi]['cm_shuffle']
        time_window_centers = cm_true_per_roi[roi]['time_window_centers']
        window_size = cm_true_per_roi[roi]['window_size']
        step_size = cm_true_per_roi[roi]['step_size']

        # Compute accuracies
        accuracies_true, accuracies_shuffle = compute_accuracies(cm_true, cm_shuffle)

        # Perform time permutation cluster test
        significant_clusters, p_values = perform_time_perm_cluster_test(
            accuracies_true, accuracies_shuffle, p_thresh=0.05, n_perm=100, seed=42
        )

        # Store significant clusters and p-values
        cm_true_per_roi[roi]['significant_clusters'] = significant_clusters
        cm_true_per_roi[roi]['p_values'] = p_values

        # Optionally, store accuracies
        cm_true_per_roi[roi]['accuracies_true'] = accuracies_true
        cm_shuffle_per_roi[roi]['accuracies_shuffle'] = accuracies_shuffle
        print(significant_clusters)

        # Plot accuracies
        plot_accuracies(
            time_points=time_window_centers,
            accuracies_true=accuracies_true,
            accuracies_shuffle=accuracies_shuffle,
            significant_clusters=significant_clusters,
            window_size=window_size,
            step_size=step_size,
            sampling_rate=sampling_rate,
            condition_comparison=condition_comparison,
            roi=roi,
            save_dir=save_dir
        )

Processing ROI: lpfc
Concatenated data shape for lpfc: (512, 59, 641)
Condition BigLetters has 83 trials
Condition BigLetterh has 100 trials
start times are: [-256, -224, -192, -160, -128, -96, -64, -32, 0, 32, 64, 96, 128, 160, 192, 224, 256, 288, 320]
Effective window size: 64
Effective step size: 32
Number of windows: 19
time_window_centers are: [-0.875, -0.75, -0.625, -0.5, -0.375, -0.25, -0.125, 0.0, 0.125, 0.25, 0.375, 0.5, 0.625, 0.75, 0.875, 1.0, 1.125, 1.25, 1.375]


x_stacked shape: (166, 59, 641)
windowed shape: (19, 166, 59, 64)


x_stacked shape: (166, 59, 641)
windowed shape: (19, 166, 59, 64)


x_stacked shape: (166, 59, 641)
windowed shape: (19, 166, 59, 64)


x_stacked shape: (166, 59, 641)
windowed shape: (19, 166, 59, 64)


...

x_stacked shape: (230, 10, 641)
windowed shape: (19, 230, 10, 64)


x_stacked shape: (230, 10, 641)
windowed shape: (19, 230, 10, 64)


x_stacked shape: (230, 10, 641)
windowed shape: (19, 230, 10, 64)




100%|██████████| 500/500 [15:43<00:00,  1.89s/it]


[False False False False False False False False False False  True  True
  True  True  True  True  True  True  True]
[False False False False False False False False False False False False
 False False False False False False False]
[False False False False False False False False False False False False
 False False False False  True  True  True]
[ True  True False False False False False False False False False False
 False  True  True False False False False]
[False False False False False False False False False False False False
  True  True  True  True  True  True  True]


In [160]:
for condition_comparison in condition_comparison_confusion_matrices:
    cm_true_per_roi = condition_comparison_confusion_matrices[condition_comparison]['cm_true_per_roi']
    for roi in cm_true_per_roi:
        significant_clusters = cm_true_per_roi[roi]['significant_clusters']
        print(f'significant clusters for {condition_comparison} and {roi} are: {significant_clusters}')

        p_values = cm_true_per_roi[roi]['p_values']
        print(f'p values for {condition_comparison} and {roi} are: {p_values}')

        accuracies_true = cm_true_per_roi[roi]['accuracies_true']
        print(f'accuracies for {condition_comparison} and {roi} are: {accuracies_true}')

        # Now you can use significant_clusters, p_values, and accuracies_true as needed

significant clusters for BigLetter and lpfc are: [False False False False False False False False False]
p values for BigLetter and lpfc are: [0.40594059 0.33663366 0.1980198  0.23762376 0.34653465 0.3960396
 0.45544554 0.56435644 0.42574257]
accuracies for BigLetter and lpfc are: [[0.51204819 0.51807229 0.48192771 0.4939759  0.51204819]
 [0.51807229 0.52409639 0.48192771 0.4939759  0.51807229]
 [0.53012048 0.52409639 0.5060241  0.4939759  0.52409639]
 [0.51807229 0.51204819 0.5060241  0.4939759  0.53012048]
 [0.5        0.51204819 0.5060241  0.4939759  0.5       ]
 [0.4939759  0.51204819 0.5        0.48192771 0.51807229]
 [0.5        0.4939759  0.5060241  0.48192771 0.51807229]
 [0.5        0.48192771 0.4939759  0.5        0.5       ]
 [0.51204819 0.5        0.5        0.5060241  0.5       ]]
significant clusters for BigLetter and v1 are: [False False False False False False False False False]
p values for BigLetter and v1 are: [0.96039604 1.         1.         1.         0.99009901 0

testing code

In [239]:
# Main code for testing with fake accuracies
import os

# Simulate condition_comparisons and rois for testing
test_condition_comparisons = {'TestCondition': ['fake_condition']}
test_rois = ['TestROI']
test_save_dir = './test_plots'
test_sampling_rate = 256

# Parameters for fake data
test_n_windows = 10
test_n_repeats = 10
test_n_permutations = 100
test_time_points = np.linspace(-1.5, 1, test_n_windows)
np.random.seed(42)

# Generate fake accuracies
test_accuracies_true = np.zeros((test_n_windows, test_n_repeats))
for i in range(test_n_windows):
    if i < test_n_windows // 2:
        test_accuracies_true[i] = 0.5 + 0.01 * np.random.randn(test_n_repeats)
    else:
        test_accuracies_true[i] = 0.6 + 0.05 * np.random.randn(test_n_repeats)
test_accuracies_true = np.clip(test_accuracies_true, 0, 1)

test_accuracies_shuffle = 0.5 + 0.05 * np.random.randn(test_n_windows, test_n_permutations)
test_accuracies_shuffle = np.clip(test_accuracies_shuffle, 0, 1)

# Insert fake accuracies into pipeline
for test_condition_comparison, test_strings_to_find in test_condition_comparisons.items():
    test_cm_true_per_roi = {}
    test_cm_shuffle_per_roi = {}

    for test_roi in test_rois:
        print(f"Processing ROI: {test_roi}")

        # Simulate the confusion matrices with fake data
        test_cm_true = np.zeros((test_n_windows, test_n_repeats, 2, 2))
        test_cm_shuffle = np.zeros((test_n_windows, test_n_permutations, 2, 2))

        # Use fake accuracies to skip confusion matrix generation
        test_time_window_centers = test_time_points
        test_window_size = 128
        test_step_size = 64

        # Perform time permutation cluster test
        test_significant_clusters, test_p_values = perform_time_perm_cluster_test(
            test_accuracies_true, test_accuracies_shuffle, p_thresh=0.05, n_perm=100, seed=42
        )

        # Print significant clusters
        print(f"Significant Clusters for {test_roi}: {test_significant_clusters}")

        # Plot accuracies
        plot_accuracies(
            time_points=test_time_window_centers,
            accuracies_true=test_accuracies_true,
            accuracies_shuffle=test_accuracies_shuffle,
            significant_clusters=test_significant_clusters,
            window_size=test_window_size,
            step_size=test_step_size,
            sampling_rate=test_sampling_rate,
            condition_comparison=test_condition_comparison,
            roi=test_roi,
            save_dir=test_save_dir
        )


Processing ROI: TestROI
Significant Clusters for TestROI: [False False False False False  True  True  True  True  True]


In [236]:
# Testing code with timepoints from -1.5 to 1
n_windows = 10  # Number of time windows
n_repeats = 10  # Number of repeats for true accuracies
n_permutations = 100  # Number of permutations for shuffled accuracies

# Generate time_points evenly spaced between -1.5 and 1
test_time_points = np.linspace(-1.5, 1, n_windows)  # 641 timepoints from -1.5 to 1 seconds

# Set a random seed for reproducibility
np.random.seed(42)

# Generate accuracies for true data
# First half at chance level (0.5), second half slightly above chance (e.g., 0.6)
accuracies_true = np.zeros((n_windows, n_repeats))
for i in range(n_windows):
    if i < n_windows // 2:
        accuracies_true[i] = 0.5 + 0.01 * np.random.randn(n_repeats)  # Near chance level
    else:
        accuracies_true[i] = 0.6 + 0.05 * np.random.randn(n_repeats)  # Slightly above chance

# Ensure accuracies are within [0, 1]
accuracies_true = np.clip(accuracies_true, 0, 1)

# Generate fake accuracies for shuffled data: around chance level (e.g., 0.5)
accuracies_shuffle = 0.5 + 0.05 * np.random.randn(n_windows, n_permutations)
accuracies_shuffle = np.clip(accuracies_shuffle, 0, 1)

# Generate significant_clusters array: mark windows 300 to 320 as significant
significant_clusters = np.zeros(n_windows, dtype=bool)
significant_clusters[4:6] = True  # Indices 4 to 6 inclusive

# Now, call the plot_accuracies function with the fake data
plot_accuracies(
    time_points=test_time_points,
    accuracies_true=accuracies_true,
    accuracies_shuffle=accuracies_shuffle,
    significant_clusters=significant_clusters,
    window_size=256,
    step_size=128,
    sampling_rate=1000,
    condition_comparison='TestCondition',
    roi='TestROI',
    save_dir='./test_plots',
)

print(f"Plot saved in directory: {save_dir}")

Plot saved in directory: C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\decoding\figs


In [222]:
x = np.arange(21)  # Example data
window_size = 5
step_size = 3
axis = 0

# Sliding window view with overlapping windows
windowed = sliding_window_view(x, window_shape=window_size)
print("original:", windowed)  # Original sliding window view

# Apply step_size
full_steps = (len(x) - window_size) // step_size + 1  # Calculate valid steps
slicing = [slice(None)] * windowed.ndim
slicing[axis] = slice(0, None, step_size)  # Use step_size in the slice
windowed = windowed[tuple(slicing)]

print("sliced: ", windowed)  # Downsampled sliding windows


original: [[ 0  1  2  3  4]
 [ 1  2  3  4  5]
 [ 2  3  4  5  6]
 [ 3  4  5  6  7]
 [ 4  5  6  7  8]
 [ 5  6  7  8  9]
 [ 6  7  8  9 10]
 [ 7  8  9 10 11]
 [ 8  9 10 11 12]
 [ 9 10 11 12 13]
 [10 11 12 13 14]
 [11 12 13 14 15]
 [12 13 14 15 16]
 [13 14 15 16 17]
 [14 15 16 17 18]
 [15 16 17 18 19]
 [16 17 18 19 20]]
sliced:  [[ 0  1  2  3  4]
 [ 3  4  5  6  7]
 [ 6  7  8  9 10]
 [ 9 10 11 12 13]
 [12 13 14 15 16]
 [15 16 17 18 19]]


In [230]:
x_data = np.arange(10)  # Example data
window_size = 3
step_size = 2
windows = windower(x_data, window_size=window_size, step_size=step_size)

print("Input data:", x_data)
print("Sliding windows:\n", windows)

Input data: [0 1 2 3 4 5 6 7 8 9]
Sliding windows:
 [[0 1 2]
 [2 3 4]
 [4 5 6]
 [6 7 8]]
